In [1]:
# import pakcages 
import pandas as pd 
import numpy as np  
import netCDF4 as nc
import os 
import matplotlib.pyplot as plt 
import datetime as dt 
import time 

### Calculate TROPOMI Days

In [4]:
def locate_file(d_str,path): 
    """
    function to locate TROPOMI level-2 data file based on datetime string  
    
    param: d_str-> datetime string (e.g., 20210101)
    param: path -> folder path for searching TROPOMI data files
    
    """
    files=[]
    for r, d, f in os.walk(path):
        for file in f:
            a = file.split('_')
            qaue_time = a[8] 
            if d_str in qaue_time:
                files.append(os.path.join(r, file))
    return files 

In [ ]:
def calculate_TROPOMI_day(rav_lon,rav_lat,start_day,end_year,files_path,qa_check,output_path):
    
    """
    function to calculate TROPOMI days for each day 
    param: rav_lon -> flatten longitudes array 
    param: rav_lat -> flatten latitude array 
    param: start_day -> datatime object of the start day
    param: end_year -> year for end the analysis 
    param: files_path -> folder path for searching TROPOMI data files
    param: qa_check -> threshold of qa value for determine TROPOMI day
    param: output_path -> folder path for output TROPOMI day 
    """
    
    odf = pd.DataFrame(data={'lon':rav_lon,
                        'lat':rav_lat})
    
    while start_day.year<end_year:
        day_str = daytime.strftime("%Y%m%d")
        day_files = locate_file(day_str,files_path)
        print(day_str,len(day_files))
        
        i = 0 
        for f in day_files:
            # read TROPOMI level-2 data product
            tropomi_data = nc.Dataset(f,'r')
            var = tropomi_data.groups['PRODUCT'].variables
            # extract qa value, longitude, and latitude
            qa = var['qa_value'][0,:,:].data
            lat = var['latitude'][0,:,:].data
            lon = var['longitude'][0,:,:].data
            tropomi_data.close()
            # find valid observations with qa value > qa threshold
            valid_qa = qa[qa>qa_check]
            valid_lat = lat[qa>qa_check]
            valid_lon = lon[qa>qa_check]
            
            # convert valid observations to dataframe
            valid_lat = np.round(valid_lat,decimals=1)
            valid_lon = np.round(valid_lon,decimals=1)
            valid_lat  = valid_lat.astype(str)
            valid_lon = valid_lon.astype(str)

            vdf = pd.DataFrame(data={'lon':valid_lon,
                                'lat':valid_lat,
                                'qa_{}'.format(i):valid_qa})
            # merge two dataframes     
            odf = pd.merge(odf, vdf, on=['lon', 'lat'],how='left')
            odf.drop_duplicates(inplace=True)
            i += 1 
        
        # calcualte TROPOMI day
        day = odf.iloc[:,2:].sum(axis=1)
        day = np.array(day)

        da = day > 0 
        db = day <= 0 
        day[da] = 1 
        day[db] = 0 
        arr = np.reshape(day,(1800,3600))
        
        # export TROPOMI day
        epath = os.path.join(output_path,"TROPOMIdays_{}.nc".format(day_str))
        TRO = nc.Dataset(epath, 'w', format='NETCDF4_CLASSIC')
        lat = TRO.createDimension('lat', 1800)
        lon = TRO.createDimension('lon', 3600)
        td = TRO.createVariable('td', int, ('lat', 'lon'))
        td[:] = arr
        TRO.close()
        
        # recreated 0.1 by 0.1 grid cell dataframe
        odf = pd.DataFrame(data={'lon':rav_lon,
                        'lat':rav_lat})

        print(f"Finished TROPOMI day for: {start_day}")
        
        daytime += dt.timedelta(days=1)

In [2]:
# define a global 0.1 x 0.1 degree grid cells
longitudes = np.arange(-180,180,0.1)
latitudes = np.arange(-90,90,0.1)
X,Y = np.meshgrid(longitudes,latitudes)

In [3]:
# convert grid cells to dataframe 
rav_lon = np.ravel(X)
rav_lat = np.ravel(Y)
rav_lat = np.round(rav_lat,decimals=1)
rav_lon = np.round(rav_lon,decimals=1)
rav_lon = rav_lon.astype(str)
rav_lat = rav_lat.astype(str)


In [ ]:
start_day = dt.datetime(2020,1,1)
end_year = 2021
files_path = ""
qa_check = 0.5 
output_path = ""
calculate_TROPOMI_day(rav_lon,rav_lat,start_day,end_year,files_path,qa_check,output_path)

### Calculate TROPOMI Observational Coverage, consecutive TROPOMI days, and maximum TROPOMI gap day

In [9]:
path  = "" # where TROPOMI days netCDF files were saved 
TROPOMI_day_files=[]
for r, d, f in os.walk(path):
    for file in f:
        if 'TROPOMI' in file:
            TROPOMI_day_files.append(os.path.join(r, file))

In [10]:
def cal_consecutive_day(data):
    longest = 0
    current = 0
    for num in data:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0

    return max(longest, current)

In [11]:
def cal_gap_day(data):
    longest = 0
    current = 0
    for num in data:
        if num == 0:
            current += 1
        else:
            longest = max(longest, current)
            current = 0

    return max(longest, current)

In [12]:
# calculate TROPOMI Observational Coverage 
Tro = np.empty(shape=(365,1800,3600),dtype=int)
i = 0 
for file in TROPOMI_day_files: 
    d = nc.Dataset(file,'r')
    td = d.variables['td'][:]
     
    Tro[i,:,:] = td 
    d.close()
    i += 1 
TRO_days  = np.sum(Tro,axis=0)

TOC = (TRO_days/365)*100

In [ ]:
# calcualte consecutive TROPOMI days and maximum TROPOMI gap day
Tro_max = np.zeros(shape=(1800,3600)) 
Tro_gap = np.zeros(shape=(1800,3600)) 
i = 0 
for i in range(1800):
    j = 0 
    for j in range (3600):
        tro_grid = Tro[:,i,j]
        # calculate consective TROPOMI days
        cm = consecutive_one(tro_grid)
        Tro_max[i,j] = cm 
        # calculate maximum gap days 
        gap = consecutive_zero(tro_grid)
        Tro_gap[i,j] = gap
        j += 1
    i += 1


In [ ]:
# Use following script to save calculated results  
TRO = nc.Dataset("your path", 'w', format='NETCDF4_CLASSIC')
lat = TRO.createDimension('lat', 1800)
lon = TRO.createDimension('lon', 3600)
td = TRO.createVariable('td', int, ('lat', 'lon'))
td[:] = TOC # Tro_max or Tro_gap 
TRO.close()